# Short Time Crest Factor Analysis

In this notebook, selected audio signals (songs) are analyzed
in terms of crest factor.
Similar to short-time Fourier analysis,
the crest factor is evaluated in a frame-by-frame fashion
where consecutive frames overlap.
The short-time analysis is performed for differently phase shifted signals.

In [ ]:
import numpy as np
import util
import soundfile as sf
import importlib
import matplotlib.pyplot as plt
from os.path import join
from matplotlib import cm
from matplotlib import gridspec

importlib.reload(util)

In [ ]:
src_dir = '../data/source-signals'
out_dir = './'
phase_angles = np.linspace(0, 2 * np.pi, endpoint=False, num=int(360))  # rad
filter_order = 3963530

filename = 'pnoise_voss_full'
x, fs = sf.read(join(src_dir, filename + '.wav'))
n_start, n_stop = 10789628, 10893326  # in samples
t_fadein, t_fadeout = 10, 10  # in milliseconds

In [ ]:
# filename = 'hotelcalifornia_mono_full'
# x, fs = sf.read(join(src_dir, filename + '.wav'))
# n_start, n_stop = 1981766, 2123080  # in samples
# t_fadein, t_fadeout = 10, 10  # in milliseconds

In [ ]:
# filename = 'showme_mono_full'
# x, fs = sf.read(join(src_dir, filename + '.wav'))
# n_start, n_stop = 4630950, 4713552  # in samples
# t_fadein, t_fadeout = 10, 10  # in milliseconds

In [ ]:
# filename = 'asphyxiation_mono_full'
# x, fs = sf.read(join(src_dir, filename + '.wav'))
# n_start, n_stop = 1734705, 1817408  # in samples
# t_fadein, t_fadeout = 10, 10  # in milliseconds

In [ ]:
# filename = 'knifeparty404_mono_full'
# songname = 'knifeparty404'
# x, fs = sf.read(join(src_dir, filename + '.wav'))
# n_start, n_stop = 10789628, 10893326  # in samples
# t_fadein, t_fadeout = 10, 10  # in milliseconds

In [ ]:
L_total = len(x)
n_fadein = util.t2n(t_fadein, fs=fs, ms=True)
n_fadeout = util.t2n(t_fadeout, fs=fs, ms=True)
frame_length = n_stop - n_start
hop_size = int(np.round(0.5 * frame_length))
n_0 = n_start // hop_size
num_frames = int((L_total - n_0 - frame_length) / hop_size) + 1
m_selection = int((n_start - n_0 - frame_length) / hop_size) + 1
time = util.n2t(n_0 + np.arange(num_frames) * hop_size, fs=fs)

In [ ]:
h0 = util.constant_phase_shifter(filter_order, 0)[1]
hH = util.constant_phase_shifter(filter_order, -0.5 * np.pi)[1]
y0 = util.acausal_filter(x, h0)
yH = util.acausal_filter(x, hH)

crest_factor = np.zeros((len(phase_angles), num_frames))
dc_bias = np.zeros_like(crest_factor)

for i, phi in enumerate(phase_angles):
    y = np.cos(phi) * y0 - np.sin(phi) * yH
    for m in range(num_frames):
        idx = n_0 + m * hop_size + np.arange(frame_length)
        y_tapered = util.fade(y[idx], n_fadein, n_fadeout)
        crest_factor[i, m] = util.crest_factor(y_tapered)
        dc_bias[i, m] = np.sum(y_tapered)

In [ ]:
fig, ax = plt.subplots()
viridis = cm.get_cmap('viridis', num_frames).colors

for m  in range(num_frames):
    ax.plot(phase_angles, util.db(crest_factor[:, m]),
            color=viridis[m], label='{}'.format(m), alpha=0.25);
ax.set_xlabel(r'$\varphi$')
ax.set_ylabel('Crest Factor / dB')
ax.set_xlim(0, np.pi)
ax.grid();

In [ ]:
def frame2sec(m, frame_length, hop_size, n_0, fs):
    return util.n2t(n_0 + m * hop_size + frame_length / 2, fs=fs)

C_min = np.min(crest_factor, axis=0)
C_max = np.max(crest_factor, axis=0)
C_mean = np.mean(crest_factor, axis=None)
cmin, cmax = util.db(crest_factor.min()), util.db(crest_factor.max())

phi_min = phase_angles[np.argmin(crest_factor, axis=0)]
phi_max = phase_angles[np.argmax(crest_factor, axis=0)]
dphi = phi_max - phi_min

idx_min = np.unravel_index(np.argmin(crest_factor, axis=None), crest_factor.shape)
idx_max = np.unravel_index(np.argmax(crest_factor, axis=None), crest_factor.shape)

t_selection = frame2sec(m_selection, frame_length, hop_size, n_0, fs)
t_min = frame2sec(idx_min[1], frame_length, hop_size, n_0, fs)
t_max = frame2sec(idx_max[1], frame_length, hop_size, n_0, fs)


fig = plt.figure(figsize=(12, 12))
gs = gridspec.GridSpec(5, 1, height_ratios=[1, 1, 1, 1, 5]) 

ax0 = plt.subplot(gs[0])
ax0.fill_between(time, util.db(C_max), util.db(C_min))
ax0.plot(t_selection, util.db(C_mean), 'y^', ms=15)
ax0.set_ylabel('$C_{min, max}$ / dB')
ax0.grid()
plt.setp(ax0.get_xticklabels(), visible=False)

ax = plt.subplot(gs[1], sharex=ax0)
ax.plot(time, util.db(C_max / C_min))
ax.plot(t_selection, 0, 'y^', ms=15)
ax.set_ylabel(r'$\Delta C$ / dB')
ax.grid()
plt.setp(ax.get_xticklabels(), visible=False)

ax = plt.subplot(gs[2], sharex=ax0)
ax.plot(time, phi_max, label=r'$\varphi_{max}$')
ax.plot(time, phi_min, label=r'$\varphi_{min}$')
ax.plot(t_selection, 0, 'yv', ms=15)
ax.set_ylim(0, np.pi)
ax.set_ylabel(r'$\varphi$')
ax.grid()
ax.legend(loc='lower right')
plt.setp(ax.get_xticklabels(), visible=False)

ax = plt.subplot(gs[3], sharex=ax0)
ax.plot(time, dphi)
ax.plot(t_selection, 0, 'yv', ms=15)
ax.set_ylim(-np.pi, np.pi)
ax.set_ylabel(r'$\Delta\varphi$')
ax.grid()
plt.setp(ax.get_xticklabels(), visible=False)

ax = plt.subplot(gs[4], sharex=ax0)
ax.imshow(util.db(crest_factor), vmin=cmin, vmax=cmax,
          extent=[time[0], time[-1], phase_angles[0], phase_angles[-1]], origin='lower')
ax.axis('tight')
for m in range(num_frames):
    tm = frame2sec(m, frame_length, hop_size, n_0, fs)
    phi_min = phase_angles[np.argmin(crest_factor[:, m])]
    phi_max = phase_angles[np.argmax(crest_factor[:, m])]
    ax.plot(tm, phi_min, 'wx')
    ax.plot(tm, phi_max, 'wo')
ax.plot(t_min, phase_angles[idx_min[0]], 'rx', ms=12)
ax.plot(t_max, phase_angles[idx_max[0]], 'ro', ms=12)
ax.plot(t_selection, 0, 'y^', ms=15)
ax.set_xlabel('Time / s')
ax.set_ylabel(r'$\varphi$')
ax.grid();

plt.tight_layout()
plt.savefig(filename + '.png')